In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle
import random
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
def generate_board_grid():
    """
    Create a 15x15 Scrabble-like grid where each cell is a letter (A-Z) or empty.
    We'll encode letters as 1–26 and empty as 0.
    """
    board = np.zeros((15, 15))
    for i in range(15):
        for j in range(15):
            if random.random() < 0.3:  # ~30% filled
                board[i, j] = ord(random.choice(letters)) - ord('A') + 1
    return board
def estimate_score_from_grid(board):
    """
    Synthetic scoring: higher average letter → higher score.
    """
    nonzero = board[board > 0]
    if len(nonzero) == 0:
        return 0
    avg_value = np.mean(nonzero)
    filled = len(nonzero)
    vowels = sum(chr(int(x) + 64) in "AEIOU" for x in nonzero)
    return 0.3 * filled + 0.4 * vowels + 0.3 * avg_value + random.uniform(0, 5)

N_SAMPLES = 1000
boards = np.array([generate_board_grid() for _ in range(N_SAMPLES)])
y = np.array([estimate_score_from_grid(board) for board in boards])

X = boards / 26.0
y = (y - y.min()) / (y.max() - y.min())

X = X.reshape((N_SAMPLES, 15, 15))

print("✅ Data shape:", X.shape, y.shape)

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, 15), return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

history = model.fit(X, y, epochs=20, batch_size=16, validation_split=0.1)
print("✅ Training complete.")

model.save("model/model_weights_lstm.h5")
print("✅ LSTM model saved to model/model_weights_lstm.h5")

tokenizer = {ch: i+1 for i, ch in enumerate(letters)}
with open("model/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer saved.")


✅ Data shape: (1000, 15, 15) (1000,)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 64)            20480     
                                                                 
 dropout (Dropout)           (None, 15, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 33985 (132.75 KB)
Trainable params: 33985 (132.75 KB)
Non-trainable params: 0 (0.00 Byte)
_____________________________________________

D:\anaconda\envs\scrabble-ai\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
